In [1]:
# import os
# os.chdir('..')
# os.getcwd()

In [1]:
import os 
import pandas as pd
import numpy as np
# from src.config import METADATA_PATH, SPLIT_PATH, NEGIBOX_PATH, CHEXPERT_PATH

%load_ext autoreload
%autoreload 2

In [15]:
from mae_mimic.config import METADATA_PATH, SPLIT_PATH, NEGIBOX_PATH, CHEXPERT_PATH, MIMIC_PATH

In [3]:
df_split = pd.read_csv(SPLIT_PATH)
df_metadata = pd.read_csv(METADATA_PATH)

In [4]:
# initialize view with a mapping from ViewPosition
VIEW_MAP = {
    'AP': 'frontal',
    'PA': 'frontal',
    'LATERAL': 'lateral',
    'LL': 'lateral',
    'LPO': 'other',
    'RAO': 'other',
    'RPO': 'other',
    'LAO': 'other',
    # the below are overwritten in some instances by manual review
    'AP AXIAL': 'other',
    'XTABLE LATERAL': 'other',
    'AP LLD': 'other',
    'PA LLD': 'other',
    'L5 S1': 'other',
    'SWIMMERS': 'other',
    'AP RLD': 'other',
    'PA RLD': 'other',
}
df_metadata['view'] = df_metadata['ViewPosition'].map(VIEW_MAP)

In [5]:
nb = pd.read_csv(NEGIBOX_PATH)
cx = pd.read_csv(CHEXPERT_PATH)

In [6]:
df_metadata[df_metadata['view'] == 'frontal'].merge(df_split, on='dicom_id')['split'].value_counts(dropna=False, normalize=True)

split
train       0.977964
test        0.013985
validate    0.008051
Name: proportion, dtype: float64

In [8]:
labels_cols = ['Atelectasis', 'Cardiomegaly',
       'Consolidation', 'Edema', 'Enlarged Cardiomediastinum', 'Fracture',
       'Lung Lesion', 'Lung Opacity',  'Pleural Effusion',
       'Pleural Other', 'Pneumonia', 'Pneumothorax']

In [10]:
def is_finding(row: pd.core.series.Series):
    return 1.0 in row.values 

In [11]:
nb['findings'] = nb.apply(lambda row: is_finding(row[labels_cols]), axis=1)

In [12]:
number_of_findings = nb['findings'].sum()
number_of_no_findings = len(nb[nb['No Finding']==1.0])

In [13]:
metadata_cols = ['dicom_id', 'subject_id', 'study_id', 'view']
nb_cols = labels_cols + ['subject_id', 'study_id', 'findings']
split_cols = ['dicom_id', 'split']

df = df_metadata[metadata_cols]\
        .merge(nb[nb_cols], on=['subject_id', 'study_id'])\
        .drop(columns=['subject_id', 'study_id'])\
        .merge(df_split[split_cols], on='dicom_id')
df = df[df.view == 'frontal']


df[['split', 'findings','dicom_id']].groupby(['split', 'findings']).count()#.apply(lambda x: x*100/x.sum())
# ['dicom_id']

dicom_id
split    findings          
test     False          801
         True          2602
train    False        96694
         True        141268
validate False          813
         True          1146

In [17]:
# unmark to save the csv
# df.to_csv(os.path.join(MIMIC_PATH, 'helpers/dicoms_with_labels_and_splits.csv'))